In [ ]:
# ==============================================================================
# PROJECT: HYDRO-CLIMATIC MONITORING OF HIGH-ANDEAN WETLANDS
# MASTER NOTEBOOK: PHYSICS-AWARE ML (STACKING), WAVELET ANALYSIS & VALIDATION
# ==============================================================================
# Author: Research Team
# Context: Submitted to Remote Sensing (MDPI) / Special Issue on Hydrology
# Description: This script implements a multi-sensor fusion approach (Sentinel-1/2 + ERA5)
#              using an Ensemble Stacking architecture to estimate Surface Hydric Status (SHS).
#              It includes Spatial Block Cross-Validation, Wavelet Spectral Analysis,
#              and Domain Adaptation (Transfer Learning) for ungauged basins.
# ==============================================================================

# ------------------------------------------------------------------------------
# 1. SETUP AND DEPENDENCY INSTALLATION
# ------------------------------------------------------------------------------
print(">>> 1. SETTING UP ENVIRONMENT...")

# --- A. SYSTEM CONFIGURATION & WARNINGS ---
import warnings
import os
import sys
import traceback

# Filter specific warnings for cleaner output
# We ignore FutureWarnings (library updates) and UserWarnings (layout adjustments)
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=UserWarning)

# Install specific scientific libraries not pre-installed in standard Colab envs
# shap: XAI interpretation
# PyWavelets: Spectral analysis
# xgboost/lightgbm: Gradient boosting backends
# lime: Local explanation
print("   -> Installing required libraries (SHAP, PyWavelets, LIME)...")
!pip install shap PyWavelets xgboost lightgbm lime --quiet

# --- B. STANDARD DATA SCIENCE LIBRARIES ---
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.patches as patches
from matplotlib.gridspec import GridSpec

# --- C. PHYSICS & SIGNAL PROCESSING ---
import pywt  # Continuous Wavelet Transform
from scipy import signal
from scipy.optimize import curve_fit

# --- D. MACHINE LEARNING MODELS & UTILS ---
# Ensembles & Regressors
from sklearn.ensemble import (RandomForestRegressor, StackingRegressor,
                              GradientBoostingRegressor, HistGradientBoostingRegressor)
from sklearn.linear_model import LinearRegression, Ridge, RidgeCV
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

# Model Selection, Metrics & Preprocessing
from sklearn.feature_selection import RFE
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.base import clone
from sklearn.model_selection import train_test_split

# --- E. EXPLAINABLE AI (XAI) ---
import shap
import lime
from lime import lime_tabular

# --- F. GOOGLE DRIVE MOUNT ---
from google.colab import drive

# ------------------------------------------------------------------------------
# 2. GLOBAL CONFIGURATION
# ------------------------------------------------------------------------------

# Reproducibility Seed
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

# Visual Style Settings
sns.set_style("whitegrid")
plt.rcParams.update({
    'figure.figsize': (12, 6),
    'figure.dpi': 150,
    'figure.constrained_layout.use': True,
    'font.family': 'sans-serif',
    'font.sans-serif': ['Arial', 'DejaVu Sans'],
    'font.size': 11,
    'axes.titlesize': 12,
    'axes.labelsize': 11,
    'legend.fontsize': 10,
    'axes.grid': True,
    'grid.alpha': 0.3
})

# Mount Google Drive (if Google Colab env)
try:
    from google.colab import drive
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

# ------------------------------------------------------------------------------
# 3. DATA PATHS CONFIGURATION
# ------------------------------------------------------------------------------
# NOTE: Ensure these CSV files are present in your Drive.
# These files should be generated using the GEE script provided in the repository.

# Path config
if IN_COLAB:
    print("Executing in Google Colab. Mounting Drive...")
    drive.mount('/content/drive')
  # Update this path to your specific project folder
  BASE_PATH = '/content/drive/MyDrive/andes-hydro-ml/data/'
else:
    print("Executing in local environment.")
    BASE_PATH = '../data/'

print(f"Base Path: {BASE_PATH}")

PATHS = {
    # Primary Basin: Full Ramis (Used for initial exploration if needed)
    'RAMIS': {
        'S1':      BASE_PATH + 'Ramis_S1.csv',
        'S2':      BASE_PATH + 'Ramis_S2.csv',
        'ERA5':    BASE_PATH + 'Ramis_ERA5_SM.csv',
        'ERA5_PR': BASE_PATH + 'Ramis_ERA5_PR.csv'
    },
    # Spatial Block: NORTH (TRAINING SET for Block-CV)
    'NORTE': {
        'S1':      BASE_PATH + 'Ramis_North_S1.csv',
        'S2':      BASE_PATH + 'Ramis_North_S2.csv',
        'ERA5':    BASE_PATH + 'Ramis_North_ERA5_SM.csv',
        'ERA5_PR': BASE_PATH + 'Ramis_North_ERA5_PR.csv'
    },
    # Spatial Block: SOUTH (VALIDATION SET for Block-CV)
    'SUR': {
        'S1':      BASE_PATH + 'Ramis_South_S1.csv',
        'S2':      BASE_PATH + 'Ramis_South_S2.csv',
        'ERA5':    BASE_PATH + 'Ramis_South_ERA5_SM.csv',
        'ERA5_PR': BASE_PATH + 'Ramis_South_ERA5_PR.csv'
    },
    # External Basin: ILAVE (TARGET for Transfer Learning)
    'ILAVE': {
        'S1':      BASE_PATH + 'Ilave_S1.csv',
        'S2':      BASE_PATH + 'Ilave_S2.csv',
        'ERA5':    BASE_PATH + 'Ilave_ERA5_SM.csv',
        'ERA5_PR': BASE_PATH + 'Ilave_ERA5_PR.csv'
    }
}

print("✅ ENVIRONMENT SETUP COMPLETED.")

In [ ]:
# ------------------------------------------------------------------------------
# 2. FUNCTIONS: DATA PROCESSING (ETL), PHYSICS-AWARE FE & MODELING UTILS
# ------------------------------------------------------------------------------

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import shap
from lime import lime_tabular
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error, r2_score

def load_csv(path):
    """
    Robust CSV loader with datetime parsing.
    Removes system columns from GEE export to ensure clean dataframes.
    """
    if path is None: return None
    try:
        df = pd.read_csv(path)
        # GEE exports usually contain 'system:time_start'
        if 'system:time_start' in df.columns:
            df['fecha'] = pd.to_datetime(df['system:time_start'])
        elif 'fecha' in df.columns:
            df['fecha'] = pd.to_datetime(df['fecha'])

        cols_drop = ['system:index', '.geo', 'system:time_start']
        df = df.drop(columns=cols_drop, errors='ignore')

        return df.sort_values('fecha').reset_index(drop=True)
    except Exception as e:
        print(f"⚠️ Error loading {path}: {e}")
        return None

def hydrological_feature_engineering(df):
    """
    Generates hydrological memory variables (API) and derivatives.
    CRITICAL FOR PHYSICS-AWARE APPROACH:
    incorporates system hysteresis (memory) into the feature space.
    """
    df = df.sort_values('fecha')

    # API (Antecedent Precipitation Index)
    # Adds vertical memory to the model:
    # - 3 days: Immediate surface wetting (pulse)
    # - 15 days: Basal saturation state (memory)
    if 'Precipitacion_ERA5' in df.columns:
        df['Precip_Acum_3d'] = df['Precipitacion_ERA5'].rolling(window=3, min_periods=1).sum()
        df['Precip_Acum_15d'] = df['Precipitacion_ERA5'].rolling(window=15, min_periods=1).sum()

    # Delta VV (Wetting/Drying trend)
    # Captures the rate of change in backscatter
    if 'VV' in df.columns:
        df['Delta_VV'] = df['VV'].diff().fillna(0)

    return df

def merge_datasets_advanced(df_s1, df_s2, df_era5_sm, df_era5_pr=None):
    """
    Continuous daily fusion with Gap-Filling strategies.
    CORRECTION: Handles duplicate dates (orbital overlaps) by averaging.
    """
    if df_s1 is None or df_era5_sm is None: return None

    # --- DUPLICATE HANDLING STEP ---
    # Sentinel-1 sometimes has overlapping orbits (Ascending/Descending) on the same day.
    # We average them to get a single daily representative value.
    df_s1 = df_s1.groupby('fecha').mean(numeric_only=True).reset_index()
    df_era5_sm = df_era5_sm.groupby('fecha').mean(numeric_only=True).reset_index()

    if df_s2 is not None:
        df_s2 = df_s2.groupby('fecha').mean(numeric_only=True).reset_index()

    if df_era5_pr is not None:
        df_era5_pr = df_era5_pr.groupby('fecha').mean(numeric_only=True).reset_index()
    # ----------------------------------------

    # 1. Master Daily Timeline (Ensures continuity even with gaps)
    min_date = df_s1['fecha'].min()
    max_date = df_s1['fecha'].max()
    master_timeline = pd.date_range(start=min_date, end=max_date, freq='D', name='fecha')
    df_master = pd.DataFrame(index=master_timeline).reset_index()

    # 2. Preprocessing (Limited Interpolation)
    # S1: Gap-filling max 7 days (Synoptic scale consistency)
    s1_daily = df_s1.set_index('fecha').reindex(master_timeline).interpolate(method='time', limit=7)

    # S2: Gap-filling max 14 days (Vegetation persistence is longer)
    if df_s2 is not None:
        s2_daily = df_s2.set_index('fecha').reindex(master_timeline).interpolate(method='time', limit=14)
    else:
        s2_daily = pd.DataFrame(index=master_timeline)

    # ERA5: Continuous (Resample to ensure daily consistency)
    era5_sm_daily = df_era5_sm.set_index('fecha').resample('D').mean().rename(columns={'volumetric_soil_water_layer_1': 'Humedad_ERA5'})

    # Precipitation
    if df_era5_pr is not None:
        era5_pr_daily = df_era5_pr.set_index('fecha').resample('D').mean().rename(columns={'total_precipitation_hourly': 'Precipitacion_ERA5'})
    else:
        era5_pr_daily = None

    # 3. Fusion (Merge)
    df_final = df_master.merge(s1_daily, on='fecha', how='left') \
                        .merge(s2_daily, on='fecha', how='left') \
                        .merge(era5_sm_daily, on='fecha', how='left')

    if era5_pr_daily is not None:
        df_final = df_final.merge(era5_pr_daily, on='fecha', how='left')

    # 4. Feature Engineering application (Physics-Aware vars)
    df_final = hydrological_feature_engineering(df_final)

    # 5. Final Cleaning (Only keep days with valid Radar and Moisture data)
    # We require at least Sentinel-1 and ERA5 to proceed.
    return df_final.dropna(subset=['VV', 'VH', 'Humedad_ERA5']).reset_index(drop=True)

def process_basin(config):
    """Full ETL Pipeline wrapper."""
    return merge_datasets_advanced(
        load_csv(config['S1']),
        load_csv(config['S2']),
        load_csv(config['ERA5']),
        load_csv(config.get('ERA5_PR')) # Can be None
    )

# ==============================================================================
# METRICS & EVALUATION UTILS
# ==============================================================================

def calculate_hydrological_metrics(y_true, y_pred):
    """
    Calculates comprehensive metrics including ubRMSE.
    Matches the standards of Beck et al. (2021).
    """
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)

    # Basic Metrics
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    bias = np.mean(y_pred - y_true)
    r2 = r2_score(y_true, y_pred)

    # unbiased RMSE (ubRMSE)
    ubrmse = np.sqrt(rmse**2 - bias**2)

    # KGE Calculation
    r = np.corrcoef(y_true, y_pred)[0, 1]
    alpha = np.std(y_pred) / np.std(y_true)
    beta = np.mean(y_pred) / np.mean(y_true)
    kge = 1 - np.sqrt((r - 1)**2 + (alpha - 1)**2 + (beta - 1)**2)

    return {'KGE': kge, 'ubRMSE': ubrmse, 'Bias': bias, 'RMSE': rmse, 'R2': r2}

# ==============================================================================
# EXPLAINABLE AI (XAI) & TRANSFER LEARNING VISUALIZATION
# ==============================================================================

def plot_shap_interaction(shap_values, X, feature_names, col_x='VV', col_interaction='NDMI'):
    """
    Plots the interaction effect between two features using constrained layout.
    """
    try:
        if col_x not in feature_names or col_interaction not in feature_names:
            return

        idx_x = feature_names.index(col_x)
        idx_int = feature_names.index(col_interaction)

        plt.figure(figsize=(10, 6), layout='constrained')
        shap.dependence_plot(idx_x, shap_values, X, feature_names=feature_names,
                             interaction_index=idx_int, show=False)
        plt.title(f'Physical Interaction: {col_x} vs {col_interaction}')
        plt.show()
    except Exception as e:
        print(f"⚠️ Could not generate interaction plot: {e}")

def plot_local_lime_analysis(model, X_train, X_instance, feature_names):
    """
    Uses LIME to explain a SINGLE specific prediction (e.g., Drought Peak).
    """
    print("   -> Generating local LIME explanation...")

    explainer = lime_tabular.LimeTabularExplainer(
        training_data=X_train,
        feature_names=feature_names,
        mode='regression',
        verbose=False,
        random_state=42
    )

    # Explain the instance
    exp = explainer.explain_instance(X_instance, model.predict, num_features=5)

    # Show plot
    fig = exp.as_pyplot_figure()
    plt.title("Local Feature Contribution (LIME) - Critical Event")
    plt.tight_layout() # LIME figure handles layout internally differently
    plt.show()

def compare_contextual_importance(shap_values_train, shap_values_drought, feature_names):
    """
    Compares variable importance: Normal Conditions vs. Drought.
    Visualizes the regime shift in the model's logic.
    """
    # Calculate mean absolute SHAP values for each feature
    imp_train = np.mean(np.abs(shap_values_train), axis=0)
    imp_drought = np.mean(np.abs(shap_values_drought), axis=0)

    df_imp = pd.DataFrame({
        'Variable': feature_names,
        'Importance_Normal': imp_train,
        'Importance_Drought': imp_drought
    }).set_index('Variable')

    # Normalize to compare proportions (relative importance)
    df_imp = df_imp / df_imp.sum()

    # Plot
    ax = df_imp.plot(kind='bar', figsize=(12, 5), color=['gray', '#d32f2f'], alpha=0.9, width=0.8)
    plt.title('Physical Regime Shift: Variable Importance (Historical vs Drought 2023)', fontweight='bold')
    plt.ylabel('Relative SHAP Importance')
    plt.xlabel('Predictor')
    plt.grid(axis='y', linestyle='--', alpha=0.5)
    plt.tight_layout()
    plt.show()

def transfer_learning_domain_adaptation(model_source, df_target, features, target_col, n_calibration=100):
    """
    Implements Transfer Learning via 'Residual Correction'.

    1. Uses Ramis model (Source) to predict on Ilave (Target).
    2. Takes a small sample (n_calibration) as 'Field Data'.
    3. Trains a light Correction Model (Ridge) to learn the local BIAS.
    4. Corrects final predictions.
    """
    # 1. Split Calibration (small) and Test (large)
    # Using fixed seed for reproducibility
    df_calib = df_target.sample(n=n_calibration, random_state=42)
    df_test = df_target.drop(df_calib.index)

    X_calib = df_calib[features].values
    y_calib = df_calib[target_col].values

    X_test = df_test[features].values
    y_test = df_test[target_col].values

    print(f"   -> Calibrating with {n_calibration} local samples (simulated field work)...")

    # 2. Base Prediction (Source Model)
    pred_base_calib = model_source.predict(X_calib)
    pred_base_test = model_source.predict(X_test)

    # 3. Calculate Residuals (Systematic Error in Target Domain)
    residuos_calib = y_calib - pred_base_calib

    # 4. Train Correction Model (Fine-Tuning)
    # Ridge is preferred over LinearRegression to avoid overfitting on small n
    correction_model = Ridge(alpha=1.0)
    correction_model.fit(X_calib, residuos_calib)

    # 5. Predict correction for the rest of data
    correccion_test = correction_model.predict(X_test)

    # 6. Final Adapted Prediction
    y_final_pred = pred_base_test + correccion_test

    return y_test, pred_base_test, y_final_pred

In [ ]:
# ------------------------------------------------------------------------------
# 3. FUNCTIONS: THEORETICAL PHYSICS (WCM) & DROUGHT INDICES
# ------------------------------------------------------------------------------

from scipy.optimize import curve_fit
from scipy import stats

def water_cloud_model(X, A, B, C, D):
    """
    Semi-empirical Water Cloud Model (WCM) equation.
    Models radar backscatter (sigma0) as a sum of vegetation scattering and
    attenuated soil moisture contribution.

    Formula:
        sigma_veg = A * V1 * cos(theta) * (1 - tau^2)
        tau^2     = exp(-2 * B * V2 / cos(theta))
        sigma_soil= C + D * SM
        sigma_tot = sigma_veg + (tau^2 * sigma_soil)

    Args:
        X (tuple): (Soil Moisture array, Vegetation Index array).
        A: Vegetation scattering parameter.
        B: Vegetation attenuation parameter.
        C: Soil intercept (calibration constant).
        D: Soil moisture sensitivity slope.

    Returns:
        np.array: Simulated Backscatter in Linear units (or dB, depending on C/D context).
    """
    soil_moisture, veg_index = X

    # Standard Incidence Angle for Sentinel-1 IW mode (~30-46 deg).
    # We use a representative mean of 38 degrees converted to radians.
    THETA_RAD = np.radians(38)

    # 1. Two-way attenuation (Transmissivity squared)
    # Models how much signal penetrates the canopy.
    tau2 = np.exp(-2 * B * veg_index / np.cos(THETA_RAD))

    # 2. Soil Contribution (Linear approximation in dB domain)
    # Represents the backscatter from bare soil.
    sigma_soil = C + D * soil_moisture

    # 3. Vegetation Contribution (Water Cloud)
    # Represents volume scattering from the canopy.
    sigma_veg = A * veg_index * np.cos(THETA_RAD) * (1 - tau2)

    # Total Backscatter
    return sigma_veg + (tau2 * sigma_soil)

def fit_water_cloud_model(df):
    """
    Fits the WCM physical parameters (A, B, C, D) using Non-Linear Least Squares.
    Used to validate that the ML model aligns with physical theory.

    Returns:
        tuple: (Optimal Parameters [A,B,C,D], R2 Score of the physical fit)
    """
    # 1. Check for required columns
    required_cols = ['Humedad_ERA5', 'NDVI', 'VV']
    if not all(col in df.columns for col in required_cols):
        print("⚠️ Missing columns for WCM (NDVI/VV/Humedad required). Skipping.")
        return None, None

    # 2. STRICT CLEANING: WCM fitting is sensitive to outliers/Infs
    # We create a temporary subset and drop ANY missing value or infinite.
    df_wcm = df[required_cols].replace([np.inf, -np.inf], np.nan).dropna()

    # 3. Data sufficiency check
    if len(df_wcm) < 50:
        print(f"⚠️ Insufficient clean data for WCM fitting (n={len(df_wcm)}). Need > 50.")
        return None, None

    # Prepare Data
    # X_data = (Soil Moisture, Vegetation)
    X_data = (df_wcm['Humedad_ERA5'].values, df_wcm['NDVI'].values)
    y_data = df_wcm['VV'].values

    try:
        # Initial guesses (p0) and physical bounds
        # A, B > 0 (Vegetation physics)
        # C in [-30, 0] (Typical dry soil backscatter in dB)
        # D > 0 (Backscatter increases with moisture)
        p0 = [0.1, 0.1, -20, 20]
        bounds = ([0, 0, -35, 0], [2, 5, -5, 60])

        popt, _ = curve_fit(
            water_cloud_model,
            X_data,
            y_data,
            p0=p0,
            bounds=bounds,
            maxfev=5000
        )

        # Calculate R2 to assess physical consistency
        y_pred = water_cloud_model(X_data, *popt)
        r2_phys = r2_score(y_data, y_pred)

        return popt, r2_phys

    except Exception as e:
        print(f"⚠️ WCM Fitting failed (Convergence error): {e}")
        return None, None

def calculate_spi(df, precip_col='Precipitacion_ERA5', scales=[3, 12]):
    """
    Calculates the Standardized Precipitation Index (SPI) approximation.
    Uses a Log-Normal distribution approach (proxy for Gamma) for computational efficiency.
    Useful for contextualizing the drought severity.
    """
    if precip_col not in df.columns:
        return None

    df_spi = df.copy()

    # Handle zeros for Log calculation (replace with epsilon)
    # ERA5 hourly precipitation sum might be 0.
    precip = df_spi[precip_col].replace(0, 0.01)

    for m in scales:
        window_days = m * 30 # Approx days per month

        # Rolling Sum (Accumulated Precipitation)
        # We require at least one full window to calculate
        acum = precip.rolling(window=window_days, min_periods=window_days).sum()

        # Log-Normal Transformation (Simple SPI)
        # Standard SPI uses Gamma, but Log-Normal is a widely accepted proxy for large N.
        log_acum = np.log(acum)

        # Standardization (Z-Score)
        # (X - Mean) / Std
        df_spi[f'SPI_{m}M'] = (log_acum - log_acum.mean()) / log_acum.std()

    return df_spi

In [ ]:
# ------------------------------------------------------------------------------
# 4. FUNCTIONS: ADVANCED MODELING (STACKING) & UNCERTAINTY QUANTIFICATION
# ------------------------------------------------------------------------------

from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestRegressor, StackingRegressor, HistGradientBoostingRegressor
from sklearn.linear_model import RidgeCV
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

def recursive_feature_selection(X, y, feature_names, n_features=5):
    """
    Performs Recursive Feature Elimination (RFE) to select the most physical drivers.
    Uses a Random Forest as the base estimator to capture non-linear importance.

    Args:
        X (array): Feature matrix.
        y (array): Target variable.
        feature_names (list): List of column names.
        n_features (int): Number of top features to select.

    Returns:
        list: Names of the selected features.
    """
    # Base estimator for importance ranking
    model = RandomForestRegressor(n_estimators=50, random_state=42, n_jobs=-1)

    rfe = RFE(estimator=model, n_features_to_select=n_features)
    rfe.fit(X, y)

    # Filter selected features
    selected = [n for n, s in zip(feature_names, rfe.support_) if s]
    rejected = [n for n, s in zip(feature_names, rfe.support_) if not s]

    print(f"   -> RFE Selected Features ({len(selected)}): {selected}")
    print(f"   -> RFE Discarded Features: {rejected}")

    return selected

def train_stacking_ensemble(X_train, y_train):
    """
    Trains the 2-Level Stacking Ensemble architecture.

    Architecture:
    - Level 0 (Base Learners):
        * Random Forest: For hierarchical structure.
        * XGBoost & LightGBM: For gradient boosting performance.
    - Level 1 (Meta-Learner):
        * RidgeCV: Regularized regression with internal Cross-Validation
          to optimally weight the base models (Bias Correction).

    Returns:
        StackingRegressor: Trained model ready for prediction.
    """
    # Level 0: Diverse Base Learners
    estimators = [
        ('rf', RandomForestRegressor(n_estimators=150, max_depth=7, random_state=42, n_jobs=-1)),
        ('xgb', XGBRegressor(n_estimators=150, learning_rate=0.05, n_jobs=-1, random_state=42)),
        ('lgbm', LGBMRegressor(n_estimators=150, learning_rate=0.05, verbose=-1, n_jobs=-1, random_state=42))
    ]

    # Level 1: Meta-Learner with built-in Cross-Validation for Alpha selection
    # alphas: Regularization strength candidates
    meta_learner = RidgeCV(alphas=[0.1, 1.0, 10.0], cv=5)

    reg = StackingRegressor(
        estimators=estimators,
        final_estimator=meta_learner,
        cv=5,       # Internal CV for stacking prediction generation
        n_jobs=-1,
        passthrough=False # Meta-learner only sees base predictions, not original features
    )

    reg.fit(X_train, y_train)
    return reg

def predict_uncertainty_intervals(X_train, y_train, X_val, confidence_level=0.90):
    """
    Generates Prediction Intervals using Quantile Regression.

    Uses HistGradientBoostingRegressor because it is highly efficient for
    quantile loss optimization on medium-sized datasets.

    Args:
        confidence_level (float): Desired confidence (e.g., 0.90 for 90% PI).

    Returns:
        tuple: (y_lower, y_upper) arrays.
    """
    # Calculate quantiles (e.g., for 90% CI -> 5% and 95%)
    alpha = 1 - confidence_level
    q_low = alpha / 2
    q_high = 1 - q_low

    # Lower Bound Model (e.g., 5th percentile)
    clf_lower = HistGradientBoostingRegressor(
        loss='quantile',
        quantile=q_low,
        random_state=42,
        early_stopping=True
    )
    clf_lower.fit(X_train, y_train)
    y_lower = clf_lower.predict(X_val)

    # Upper Bound Model (e.g., 95th percentile)
    clf_upper = HistGradientBoostingRegressor(
        loss='quantile',
        quantile=q_high,
        random_state=42,
        early_stopping=True
    )
    clf_upper.fit(X_train, y_train)
    y_upper = clf_upper.predict(X_val)

    return y_lower, y_upper

In [ ]:
# ------------------------------------------------------------------------------
# 5. VISUALIZATION FUNCTIONS (PUBLICATION QUALITY)
# ------------------------------------------------------------------------------

import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import pywt
import numpy as np
import pandas as pd
from scipy import signal

# Note: These functions rely on 'calculate_hydrological_metrics' defined in Cell 2.

def plot_wavelet_analysis(time_series, title="Wavelet Scalogram", power_limit=None):
    """
    Generates the Continuous Wavelet Transform (CWT) Scalogram.
    Optimized for 'constrained_layout' (set in Cell 1).
    """
    # 1. Preprocessing
    # Resample to weekly mean to reduce high-frequency noise
    ts = time_series.resample('7D').mean()
    # Gap-filling: Interpolate small gaps, fill large ones with mean (flat signal)
    ts_filled = ts.interpolate(method='time', limit=4).fillna(ts.mean())

    # Check for Infs
    if np.isinf(ts_filled.values).any():
        ts_filled = ts_filled.replace([np.inf, -np.inf], ts.mean())

    # Remove mean but keep variance (Amplitude matters for this analysis)
    data = ts_filled.values - np.mean(ts_filled.values)

    # 2. Wavelet Transform (Morlet)
    scales = np.arange(1, 120)
    coef, freqs = pywt.cwt(data, scales, 'morl', sampling_period=1)
    power = (abs(coef)) ** 2
    period = 1. / freqs

    # Cone of Influence (COI) Mask
    coi = np.zeros((len(scales), len(data)))
    for i, p in enumerate(period):
        lim = int(p * np.sqrt(2))
        coi[i, :lim] = 1
        coi[i, -lim:] = 1

    # 3. Plotting
    fig = plt.figure(figsize=(14, 7))
    gs = GridSpec(1, 4, figure=fig)

    # --- PANEL A: SCALOGRAM ---
    ax1 = fig.add_subplot(gs[0, :3])

    # Scale color to Historical Peak if provided
    if power_limit is None:
        vmax = np.percentile(power, 99)
    else:
        vmax = power_limit

    levels = np.linspace(0, vmax, 60)

    # Plot Contours
    # extend='both' ensures values outside range are colored (saturated)
    ax1.contourf(np.arange(len(data)), period, power, levels=levels, cmap='jet', extend='both')
    # Plot COI (hatched area)
    ax1.contourf(np.arange(len(data)), period, coi, levels=[0.5, 1.5], colors='none', hatches=['//'])

    # Drought Marker (2023)
    if isinstance(time_series.index, pd.DatetimeIndex):
        start_date = time_series.index.min()
        drought_start = pd.Timestamp('2023-01-01')
        if drought_start > start_date:
            idx_2023 = int((drought_start - start_date).days / 7)
            if 0 <= idx_2023 < len(data):
                ax1.axvline(idx_2023, c='white', ls='--', lw=2, alpha=0.8)
                ax1.text(idx_2023 + 2, period.max()*0.8, "DROUGHT PERIOD",
                         color='white', fontweight='bold', rotation=90, fontsize=9)

    # Formatting
    ax1.set_yscale('log')
    ax1.set_yticks([13, 26, 52, 104])
    ax1.set_yticklabels(['Quarterly', 'Biannual', 'Annual', 'Biennial'])
    ax1.set_title(title, fontweight='bold')
    ax1.set_ylabel('Period (Weeks)')
    ax1.set_xlabel('Time (Weeks)')

    # --- PANEL B: GLOBAL SPECTRUM ---
    ax2 = fig.add_subplot(gs[0, 3], sharey=ax1)
    gws = np.mean(power, axis=1)
    ax2.plot(gws, period, c='b')
    ax2.fill_betweenx(period, 0, gws, color='b', alpha=0.2)
    ax2.set_title('Global Spectrum')
    plt.setp(ax2.get_yticklabels(), visible=False)
    ax2.grid(True, which='both', linestyle='--', alpha=0.5)

    plt.show()

def plot_uncertainty_series(y_true, y_pred, y_low, y_up, dates, title):
    """
    Plots the time series with 90% Prediction Intervals.
    Displays comprehensive metrics (KGE, ubRMSE, Bias) in the title.
    """
    # Calculate metrics
    mets = calculate_hydrological_metrics(y_true, y_pred)

    plt.figure(figsize=(14, 5))

    # 90% Confidence Band
    plt.fill_between(dates, y_low, y_up, color='gray', alpha=0.3, label='90% Prediction Interval')

    # Series
    plt.plot(dates, y_true, 'k-', lw=1, label='Observed (ERA5)', alpha=0.7)
    plt.plot(dates, y_pred, 'r--', lw=1.5, label='Estimated (Stacking)')

    # Title with Metrics
    title_text = (f"{title}\n"
                  f"KGE: {mets['KGE']:.2f} | ubRMSE: {mets['ubRMSE']:.3f} | Bias: {mets['Bias']:.3f}")

    plt.title(title_text, fontweight='bold')
    plt.xlabel('Date')
    plt.ylabel('Volumetric Soil Moisture ($m^3/m^3$)')

    plt.legend(loc='upper right', fancybox=True, framealpha=0.9)
    plt.grid(True, linestyle=':', alpha=0.6)
    plt.show()

def plot_scatter_validation(y_true, y_pred, title, color='teal'):
    """
    Generates a 1:1 Scatter Plot with Identity Line and R2 score.
    """
    mets = calculate_hydrological_metrics(y_true, y_pred)

    plt.figure(figsize=(6, 6))

    # Scatter points
    plt.scatter(y_true, y_pred, alpha=0.5, c=color, edgecolor='k', s=40)

    # 1:1 Identity Line
    min_val = min(y_true.min(), y_pred.min())
    max_val = max(y_true.max(), y_pred.max())
    plt.plot([min_val, max_val], [min_val, max_val], 'r--', lw=2, label='1:1 Identity')

    # Annotations
    plt.title(f"{title}\n$R^2={mets['R2']:.3f} | Bias={mets['Bias']:.3f}$", fontweight='bold')
    plt.xlabel('Observed (ERA5)')
    plt.ylabel('Predicted (Stacking)')

    plt.legend(loc='upper left')
    plt.grid(True, linestyle='--', alpha=0.5)
    plt.show()

def quantify_seasonality_strength(time_series, period_range=(45, 60)):
    """
    Calculates the Peak Seasonal Energy (95th percentile).
    Quantifies the failure to reach peak recharge levels.
    """
    # 1. Preprocessing
    ts = time_series.resample('7D').mean()
    ts_filled = ts.interpolate(method='time', limit=4).fillna(ts.mean())
    data = signal.detrend(ts_filled.values)

    # 2. Wavelet
    scales = np.arange(1, 120)
    coef, freqs = pywt.cwt(data, scales, 'morl', sampling_period=1)
    power = (abs(coef)) ** 2
    period = 1. / freqs

    # 3. Extract Annual Band
    mask_annual = (period >= period_range[0]) & (period <= period_range[1])
    sap_annual = np.mean(power[mask_annual, :], axis=0)
    sap_series = pd.Series(sap_annual, index=ts_filled.index)

    # 4. STATISTICAL COMPARISON (PEAK POWER)
    # We compare the 95th Percentile (The "Recharge Peaks")
    baseline_series = sap_series[sap_series.index.year < 2023]
    drought_series = sap_series[sap_series.index.year == 2023]

    baseline_peak = np.percentile(baseline_series, 95)
    drought_peak = np.percentile(drought_series, 95)

    # Percent Drop in Peak Capacity
    drop_pct = ((baseline_peak - drought_peak) / baseline_peak) * 100

    print(f"      [DEBUG] Baseline Peak (95th): {baseline_peak:.4f}")
    print(f"      [DEBUG] Drought Peak (95th): {drought_peak:.4f}")

    return {
        'Baseline_Power': baseline_peak,
        'Drought_Power': drought_peak,
        'Drop_Percentage': drop_pct
    }

def plot_interannual_energy_scan(time_series, period_range=(45, 60)):
    """
    Scans the entire timeline year-by-year to validate if the 2023 drop
    is truly unique or a recurring artifact.
    """
    # 1. Continuous Wavelet Transform (Full Series)
    ts = time_series.resample('7D').mean()
    ts_filled = ts.interpolate(method='time', limit=4).fillna(ts.mean())
    data = signal.detrend(ts_filled.values)

    scales = np.arange(1, 120)
    coef, freqs = pywt.cwt(data, scales, 'morl', sampling_period=1)
    power = (abs(coef)) ** 2
    period = 1. / freqs

    # 2. Extract Annual Energy Band
    mask_annual = (period >= period_range[0]) & (period <= period_range[1])
    sap_continuous = np.mean(power[mask_annual, :], axis=0)
    sap_series = pd.Series(sap_continuous, index=ts_filled.index)

    # 3. Group by Year (95th Percentile Peaks)
    annual_peaks = sap_series.groupby(sap_series.index.year).quantile(0.95)

    # 4. Statistical Thresholds
    mean_energy = annual_peaks.mean()
    std_energy = annual_peaks.std()
    anomaly_threshold = mean_energy - (1.5 * std_energy) # 1.5 Sigma

    # 5. Plotting
    plt.figure(figsize=(10, 5))

    colors = ['#b0bec5' if x > anomaly_threshold else '#d32f2f' for x in annual_peaks.values]
    bars = plt.bar(annual_peaks.index, annual_peaks.values, color=colors, edgecolor='k', alpha=0.8)

    plt.axhline(mean_energy, color='k', linestyle='--', label=f'Mean Historic Energy ({mean_energy:.2f})')
    plt.axhline(anomaly_threshold, color='r', linestyle=':', linewidth=2, label=r'Anomaly Threshold (-1.5$\sigma$)')

    for bar in bars:
        height = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2., height,
                 f'{height:.2f}',
                 ha='center', va='bottom', fontsize=9)

    plt.title('Interannual Scan: Peak Hydrological Cycle Energy (2016-2024)', fontweight='bold')
    plt.ylabel('Peak Spectral Power (95th Pct)')
    plt.xlabel('Year')
    plt.legend()
    plt.grid(True, axis='y', linestyle='--', alpha=0.5)
    plt.show()

    return annual_peaks

In [ ]:
# ------------------------------------------------------------------------------
# 6. MAIN WORKFLOW: MODELING, SPATIAL VALIDATION & EXPLAINABILITY
# ------------------------------------------------------------------------------

import matplotlib.dates as mdates

print("\n>>> INITIALIZING MAIN WORKFLOW (SPATIAL BLOCK-CV & TRANSFER LEARNING)...")

try:
    # --- PHASE 1: DATA LOADING & PREPROCESSING (TRAINING BLOCK) ---
    print("\n[PHASE 1] Loading Primary Training Data (Ramis North)...")

    # Note: 'PATHS' keys must match Cell 1 definition.
    df_north = process_basin(PATHS['NORTE'])

    if df_north is not None:
        print(f"✅ Ramis North Loaded: {len(df_north)} records.")

        # --- A. MODEL TRAINING (STACKING ENSEMBLE) ---
        print("\n>>> A. ADVANCED MODELING (STACKING ENSEMBLE)...")

        # 1. Feature Definition
        all_feats = ['VV', 'VH', 'NDMI', 'NDVI', 'NDWI', 'Precip_Acum_3d', 'Precip_Acum_15d', 'Delta_VV']
        avail_feats = [f for f in all_feats if f in df_north.columns]

        # 2. Temporal Split (Train < 2023, Val = 2023)
        df_north['year'] = df_north['fecha'].dt.year
        df_north = df_north.dropna(subset=avail_feats).reset_index(drop=True)

        mask_train = df_north['year'] < 2023
        mask_val_drought = df_north['year'] == 2023

        # 3. Recursive Feature Elimination (RFE)
        # Performed strictly on Training Data to avoid data leakage
        top_feats = recursive_feature_selection(
            df_north.loc[mask_train, avail_feats].values,
            df_north.loc[mask_train, 'Humedad_ERA5'].values,
            avail_feats,
            n_features=5
        )

        # Prepare matrices
        X_full = df_north[top_feats].values
        y_full = df_north['Humedad_ERA5'].values

        X_train = X_full[mask_train]
        y_train = y_full[mask_train]

        # 4. Train Model
        print(f"   -> Training Stacking Ensemble on {len(X_train)} samples...")
        model = train_stacking_ensemble(X_train, y_train)

        # --- B. PHYSICAL DIAGNOSTICS (WAVELETS & WCM) ---
        print("\n>>> B. PHYSICAL DIAGNOSTICS & CONSISTENCY CHECK...")

        # 1. WCM Fitting
        popt, r2_wcm = fit_water_cloud_model(df_north)
        if popt is not None:
            print(f"   -> WCM Fit (Physical Consistency): R²={r2_wcm:.3f} | Params: {popt}")

        # 2. Wavelet Analysis on MODEL PREDICTION
        print("   -> Analyzing seasonality breakdown on Model Predictions...")

        # Generate predictions for the full timeline to analyze the whole series
        y_pred_full = model.predict(X_full)
        ts_pred = pd.Series(y_pred_full, index=df_north['fecha']).sort_index()

        # 3. INTERANNUAL SCAN
        print("\n   -> Running Interannual Variability Scan (Checking for false positives)...")
        # This generates Figure 8 (Bar plot)
        annual_energy = plot_interannual_energy_scan(ts_pred)

        # 4. WAVELET SCALOGRAM
        # We define "Red" (Max Power) based on the STRONGEST historical year.
        ts_historic = ts_pred[ts_pred.index.year < 2023]

        # Calculate power of history manually to find the limit
        data_hist = ts_historic.resample('7D').mean().interpolate(limit=4).fillna(ts_historic.mean()).values
        data_hist = signal.detrend(data_hist)
        scales = np.arange(1, 120)
        coef, _ = pywt.cwt(data_hist, scales, 'morl', sampling_period=1)
        power_hist = (abs(coef)) ** 2
        historical_peak_power = np.max(power_hist)

        print(f"      Historical Peak Power (Baseline): {historical_peak_power:.2f}")

        plot_wavelet_analysis(ts_pred,
                              title="Spectral Power Density (Seasonality Attenuation)",
                              power_limit=historical_peak_power)

        # 5. QUANTITATIVE SEASONALITY METRICS
        print("\n   -> Quantifying Energy Loss in the Annual Cycle (45-60 weeks)...")
        stats_wavelet = quantify_seasonality_strength(ts_pred)

        print(f"      STATS REPORT:")
        print(f"      - Historical Energy: {stats_wavelet['Baseline_Power']:.4f}")
        print(f"      - Drought 2023 Energy: {stats_wavelet['Drought_Power']:.4f}")
        print(f"      - MAGNITUDE OF COLLAPSE: {stats_wavelet['Drop_Percentage']:.2f}% Reduction")

        # --- C. TEMPORAL VALIDATION: DROUGHT ANALYSIS & UNCERTAINTY ---
        print("\n>>> C. TEMPORAL VALIDATION: DROUGHT ANALYSIS & UNCERTAINTY...")
        print("   -> Calculating prediction intervals (90% CI)...")

        # 1. Prepare Plotting Data
        df_plot = df_north.sort_values('fecha').copy()
        dates_plot = df_plot['fecha']
        y_true_plot = df_plot['Humedad_ERA5'].values
        X_plot = df_plot[top_feats].values

        # 2. Generate Uncertainty Intervals (Quantile Regression)
        y_low, y_up = predict_uncertainty_intervals(X_train, y_train, X_plot, confidence_level=0.90)

        # 3. Calculate Global Metrics
        mets = calculate_hydrological_metrics(y_true_plot, y_pred_full)

        # 4. Render
        fig, ax = plt.subplots(figsize=(14, 6)) # DPI set globally in Cell 1

        # Confidence Bands
        ax.fill_between(dates_plot, y_low, y_up, color='#b0bec5', alpha=0.4,
                        label='Uncertainty (90% PI)', zorder=1)

        # Observations (ERA5)
        ax.plot(dates_plot, y_true_plot, color='black', linestyle='-', linewidth=1.2,
                label='Reference (ERA5-Land)', alpha=0.8, zorder=2)

        # Predictions (Stacking)
        ax.plot(dates_plot, y_pred_full, color='#d32f2f', linestyle='--', linewidth=1.5,
                label='Prediction (Stacking)', zorder=3)

        # Highlight Drought 2023
        start_drought = pd.to_datetime('2023-01-01')
        end_drought = pd.to_datetime('2023-12-31')
        ax.axvspan(start_drought, end_drought, color='#fff9c4', alpha=0.5, zorder=0)

        # Annotate Physical Breakdown
        ax.annotate('2023 Water Anomaly\n(Seasonality Attenuation)',
                    xy=(pd.to_datetime('2023-06-15'), 0.15),
                    xytext=(pd.to_datetime('2021-06-01'), 0.05),
                    arrowprops=dict(facecolor='black', shrink=0.05, width=1.5, headwidth=8),
                    fontsize=10, fontweight='bold', color='#bf360c',
                    bbox=dict(boxstyle="round,pad=0.3", fc="white", ec="#bf360c", alpha=0.9))

        # Metrics Info Box
        info_box = (f"Figure 5. Global Metrics (2016-2024):\n"
                    f"KGE = {mets['KGE']:.2f}\n"
                    f"ubRMSE = {mets['ubRMSE']:.3f} m³/m³\n"
                    f"Bias = {mets['Bias']:.3f}")

        ax.text(0.02, 0.95, info_box, transform=ax.transAxes, fontsize=10,
                verticalalignment='top', bbox=dict(boxstyle='round', facecolor='white', alpha=0.9))

        # Styling
        ax.set_ylabel('Volumetric Soil Moisture ($m^3/m^3$)', fontsize=12, fontweight='bold')
        ax.set_xlabel('Time (Years)', fontsize=12)
        ax.set_title('Temporal Dynamics of Surface Hydric Status and Response to Extreme Drought',
                     fontsize=14, fontweight='bold', pad=15)

        ax.xaxis.set_major_locator(mdates.YearLocator())
        ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y'))
        ax.set_ylim(0, 0.6)
        ax.set_xlim(dates_plot.min(), dates_plot.max())
        ax.legend(loc='upper right', frameon=True, fancybox=True, framealpha=0.9, fontsize=10)

        plt.show()

        # --- D. EXPLAINABILITY (XAI) ---
        print("\n>>> D. ADVANCED EXPLAINABILITY (XAI)...")

        # 1. SHAP (Surrogate Model)
        rf_explainer = RandomForestRegressor(n_estimators=100, max_depth=10, random_state=42)
        rf_explainer.fit(X_train, y_train)
        explainer = shap.TreeExplainer(rf_explainer)

        # Compute SHAP values for Drought period vs History
        X_val_drought = X_full[mask_val_drought]
        shap_vals_drought = explainer.shap_values(X_val_drought)
        shap_vals_train = explainer.shap_values(X_train)

        # 2. Contextual Comparison
        compare_contextual_importance(shap_vals_train, shap_vals_drought, top_feats)

        # 3. LIME Day Zero Analysis
        print("   -> Performing 'Day Zero' Autopsy (LIME)...")
        # Find the driest predicted day in 2023
        y_pred_23 = y_pred_full[mask_val_drought]
        idx_min = np.argmin(y_pred_23)

        X_worst_day = X_val_drought[idx_min]
        worst_date = df_north.loc[mask_val_drought, 'fecha'].iloc[idx_min].date()

        print(f"      Critical Date: {worst_date}")
        print(f"      Predicted SHS: {y_pred_23[idx_min]:.3f} m3/m3")

        plot_local_lime_analysis(model, X_train, X_worst_day, top_feats)

        # --- E. SPATIAL VALIDATION 1: INTRA-BASIN (SOUTH BLOCK) ---
        print("\n>>> E. SPATIAL CROSS-VALIDATION (BLOCK: RAMIS SOUTH)...")
        df_south = process_basin(PATHS['SUR'])

        if df_south is not None:
            # Clean NaNs in Target Domain
            df_south = df_south.dropna(subset=top_feats).reset_index(drop=True)

            if len(df_south) > 150:
                y_test_s, y_naive_s, y_adapted_s = transfer_learning_domain_adaptation(
                    model, df_south, top_feats, 'Humedad_ERA5', n_calibration=100
                )
                # Comparative Naive Transfer vs Fine-Tuned & Single Scatter for Intra-basin
                plot_transfer_results(y_test_s, y_naive_s, y_adapted_s)
                plot_scatter_validation(y_test_s, y_adapted_s, "Spatial Validation: South Block (Fine-Tuned)", color='teal')
            else:
                print("⚠️ Insufficient data in South Block.")

        # --- F. SPATIAL VALIDATION 2: INTER-BASIN (ILAVE) ---
        print("\n>>> F. REGIONAL SCALABILITY (TARGET: ILAVE BASIN)...")
        df_ilave = process_basin(PATHS['ILAVE'])

        if df_ilave is not None:
            # Clean NaNs in Target Domain
            df_ilave = df_ilave.dropna(subset=top_feats).reset_index(drop=True)

            if len(df_ilave) > 150:
                y_test_i, y_naive_i, y_adapted_i = transfer_learning_domain_adaptation(
                    model, df_ilave, top_feats, 'Humedad_ERA5', n_calibration=100
                )
                # Comparative Naive Transfer vs Fine-Tuned & Single Scatter for Inter-basin
                plot_transfer_results(y_test_i, y_naive_i, y_adapted_i)
                plot_scatter_validation(y_test_i, y_adapted_i, "Spatial Validation: Ilave Transfer (Fine-Tuned)", color='teal')
            else:
                print("⚠️ Insufficient data in Ilave Basin.")
        else:
            print("⚠️ Ilave dataset not found. Skipping Regional Validation.")

    else:
        print("❌ CRITICAL: Primary Training Data (Ramis North) failed to load.")

except Exception as e:
    print(f"\n❌ CRITICAL ERROR in Main Workflow: {e}")
    import traceback
    traceback.print_exc()